In [ ]:
# Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [ ]:
# Parametros de seguridad
umbral_pico = 0.4

max_i_r_ple1 = 17
max_i_s_ple1 = 20
max_i_t_ple1 = 17

max_i_r_ple7 = 13
max_i_s_ple7 = 30
max_i_t_ple7 = 35

tiempo_min = pd.Timedelta(minutes=5)
cant_min_picos = 10
tiempo_min_picos = pd.Timedelta(minutes=60)



In [ ]:
# Diccionario de parametros de seguridad

limites_corriente = {
    'PLE1': {
        'R': max_i_r_ple1,
        'S': max_i_s_ple1,
        'T': max_i_t_ple1,
    },
    'PLE7': {
        'R': max_i_r_ple7,
        'S': max_i_s_ple7,
        'T': max_i_t_ple7,
    }
}

In [ ]:
# Abrir los graficos en el navegador

pio.renderers.default = 'browser'

In [ ]:
# Funcion para cargar datos

from glob import glob
import pandas as pd

def cargar_maquina(base_path, maquina):
    paths = glob(f"{base_path}/{maquina}/**/*.csv", recursive=True)

    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        df['maquina'] = maquina
        dfs.append(df)

    return (
        pd.concat(dfs, ignore_index=True)
          .sort_values('temporal_placa')
          .reset_index(drop=True)
    )


In [ ]:
# Carga de datos

df_ple1 = cargar_maquina("../data", "PLE1")
df_ple7 = cargar_maquina("../data", "PLE7")

In [ ]:
# Funcion preparar_df

def preparar_df(df):
    df = df.copy()

    # Timestamp
    df['temporal_placa'] = pd.to_datetime(df['temporal_placa'])
    df['hora'] = df['temporal_placa'].dt.hour
    df['minuto'] = df['temporal_placa'].dt.minute

    # Turnos
    def asignar_turno(hora, minuto):
        t = hora * 60 + minuto

        # Pausas
        if 12*60 <= t < 12*60 + 30:
            return 'ALMUERZO'
        if 22*60 <= t < 22*60 + 30:
            return 'CENA'

        # Turnos
        if 5*60 <= t < 17*60:
            return 'TURNO MAÑANA'
        if 17*60 <= t < 22*60:
            return 'TURNO TARDE'
        if (t >= 22*60 + 30) or (t < 1*60):
            return 'TURNO TARDE'

        return 'FUERA_TURNO'

    df['turno'] = df.apply(
        lambda x: asignar_turno(x['hora'], x['minuto']),
        axis=1
    )
        
    # Potencias totales por timestamp
    df['p_activa_total'] = (
        df['potencia_a_r'] +
        df['potencia_a_s'] +
        df['potencia_a_t']
    )

    df['q_reactiva_total'] = (
        df['potencia_r_r'] +
        df['potencia_r_s'] +
        df['potencia_r_t']
    )

    return df


In [ ]:
# Ajuste de datos de las plegadoras

df_ple1 = preparar_df(df_ple1)
df_ple7 = preparar_df(df_ple7)

df_all = pd.concat([df_ple1, df_ple7], ignore_index=True)

# Filtra los datos correspondientes al 9-1-2026
import datetime as dt

dia = dt.date(2026, 1, 9)
df_all = df_all[df_all['temporal_placa'].dt.date == dia]

In [ ]:
# Calculo la media de las corrientes excluyendo los picos

def media_sin_picos(s, q = 0.9):
    return s[s <= s.quantile(q)].mean()

medias_por_fase = (
    df_all
    .groupby('maquina')
    .agg(
        i_r_media=('corriente_r', media_sin_picos),
        i_s_media=('corriente_s', media_sin_picos),
        i_t_media=('corriente_t', media_sin_picos),
    )
)

medias_por_fase

In [ ]:
# Funcion detectar_picos

def detectar_picos(signal, timestamp, umbral_pico):
    dt = timestamp.diff().dt.total_seconds()
    valid = dt > 0

    dI_dt = signal.diff() / dt

    return (
        valid &
        dI_dt.notna() &
        (dI_dt > umbral_pico)
    )

In [ ]:
# Detección de picos en las 3 fases

import pandas as pd

maquinas = df_all['maquina'].unique()

fase_cols = {
    'R': 'corriente_r',
    'S': 'corriente_s',
    'T': 'corriente_t'
}

df_peaks_list = []

for maq in maquinas:

    g = (
        df_all[df_all['maquina'] == maq]
        .sort_values('temporal_placa')
        .copy()
    )

    if g.empty:
        continue

    for fase, col_corriente in fase_cols.items():

        señal = g[col_corriente]
        timestamp = g['temporal_placa']

        idx_picos = detectar_picos(señal, timestamp, umbral_pico)

        if len(idx_picos) == 0:
            continue

        df_peaks = g.loc[idx_picos, ['temporal_placa']].copy()
        df_peaks['maquina'] = maq
        df_peaks['fase'] = fase    

        df_peaks_list.append(df_peaks)

# DataFrame final de picos
df_peaks_all = pd.concat(df_peaks_list, ignore_index=True)

df_peaks_all['temporal_placa'] = (
    pd.to_datetime(df_peaks_all['temporal_placa'], errors='coerce')
    .dt.tz_localize(None)
)


In [ ]:
# Visualizacion de picos detectados y su marca temporal

df_peaks_all = df_peaks_all.sort_values(
    by=['maquina', 'fase']
).reset_index(drop=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df_peaks_all

In [ ]:
# Analisis de Eventos

import pandas as pd

SILENCIO_MAX = pd.Timedelta(minutes=0.7)
GAP_PICO = pd.Timedelta(seconds=5)

rows = []

for (maq, fase), g in df_peaks_all.groupby(['maquina', 'fase']):

    g = g.sort_values('temporal_placa').reset_index(drop=True)

    if g.empty:
        continue

    # -------------------------------------------------
    # 1) Agrupar picos consecutivos en bloques
    # -------------------------------------------------
    g['dt_pico'] = g['temporal_placa'].diff()

    g['bloque_id'] = (
        g['dt_pico'].isna() |
        (g['dt_pico'] > GAP_PICO)
    ).cumsum()

    # -------------------------------------------------
    # 2) Resumir bloques
    # -------------------------------------------------
    bloques = (
        g.groupby('bloque_id')
        .agg(
            t_ini=('temporal_placa', 'min'),
            t_fin=('temporal_placa', 'max'),
            n_muestras=('temporal_placa', 'count')
        )
        .sort_values('t_ini')
        .reset_index()
    )

    bloques['dt_prev'] = bloques['t_ini'] - bloques['t_fin'].shift(1)
    bloques['dt_next'] = bloques['t_ini'].shift(-1) - bloques['t_fin']

    # -------------------------------------------------
    # 3) Definir bloque espurio
    # -------------------------------------------------
    bloques['bloque_espurio'] = (
        (bloques['n_muestras'] <= 3) &
        (bloques['dt_prev'] > 3 * SILENCIO_MAX) &
        (bloques['dt_next'] > 3 * SILENCIO_MAX)
    )

    bloques['bloque_espurio'] = bloques['bloque_espurio'].fillna(False)

    # -------------------------------------------------
    # 4) Volver a g y filtrar
    # -------------------------------------------------
    g = g.merge(
        bloques[['bloque_id', 'bloque_espurio']],
        on='bloque_id',
        how='left'
    )

    g_filtrado = g[~g['bloque_espurio']].copy()

    if g_filtrado.empty:
        continue

    # -------------------------------------------------
    # 5) Detectar eventos (TU lógica original)
    # -------------------------------------------------
    g_filtrado['dt'] = g_filtrado['temporal_placa'].diff()

    g_filtrado['nuevo_evento'] = (
        g_filtrado['dt'].isna() |
        (g_filtrado['dt'] > SILENCIO_MAX)
    )

    g_filtrado['evento_id'] = g_filtrado['nuevo_evento'].cumsum()

    # -------------------------------------------------
    # 6) Resumir eventos
    # -------------------------------------------------
    for evento_id, c in g_filtrado.groupby('evento_id'):

        fecha_inicio = c['temporal_placa'].iloc[0]
        fecha_fin = c['temporal_placa'].iloc[-1]

        rows.append({
            'maquina': maq,
            'fase': fase,
            'ID_Evento': evento_id,
            'Fecha Inicio': fecha_inicio,
            'Fecha Fin': fecha_fin,
            'Duracion Evento': fecha_fin - fecha_inicio
        })

# -------------------------------------------------
# Resultado final
# -------------------------------------------------
df_evento = (
    pd.DataFrame(rows)
    .sort_values(by=['maquina', 'fase', 'ID_Evento'])
    .reset_index(drop=True)
)

df_evento


In [ ]:
# Graficas de eventos por fase 

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def xref_subplot(row):
    return 'x' if row == 1 else f'x{row}'

def yref_subplot(row):
    return 'y domain' if row == 1 else f'y{row} domain'


map_fase_col = {
    'R': 'corriente_r',
    'S': 'corriente_s',
    'T': 'corriente_t'
}

colores_fase = {
    'R': 'red',
    'S': 'green',
    'T': 'blue'
}

maquinas = sorted(df_all['maquina'].unique())

# Figura con 2 filas (una por máquina)
fig = make_subplots(
    rows=len(maquinas),
    cols=1,
    shared_xaxes=False,
    subplot_titles=[f'Máquina {m}' for m in maquinas]
)

# Guardamos índices de traces por (maquina, fase)
trace_idx = {}

# -----------------
# Corrientes
# -----------------
for row, maq in enumerate(maquinas, start=1):

    df_m = df_all[df_all['maquina'] == maq]

    for fase, col in map_fase_col.items():

        visible = (fase == 'R')  # fase inicial

        fig.add_trace(
            go.Scatter(
            x=df_m['temporal_placa'],
            y=df_m[col],
            mode='lines',
            name=f'Fase {fase}',          # nombre genérico
            line=dict(color=colores_fase[fase]),
            visible=visible,
            showlegend=(row == 3)         
        ),
    row=row,
    col=1
)


        trace_idx[(maq, fase)] = len(fig.data) - 1

# Shapes (eventos) por fase
shapes_por_fase = {}

for fase in map_fase_col.keys():

    shapes = []

    for row, maq in enumerate(maquinas, start=1):

        df_e = df_evento[
            (df_evento['maquina'] == maq) &
            (df_evento['fase'] == fase)
        ]

        for _, ev in df_e.iterrows():

            shapes.append(
                dict(
                    type='rect',
                     xref=xref_subplot(row),    
                    yref=yref_subplot(row),
                    x0=ev['Fecha Inicio'],
                    x1=ev['Fecha Fin'],
                    y0=0,
                    y1=1,
                    fillcolor=colores_fase[fase],
                    opacity=0.15,
                    line_width=0
                )
            )

    shapes_por_fase[fase] = shapes

# Dropdown único
botones = []

for fase in map_fase_col.keys():

    visibles = [False] * len(fig.data)

    for maq in maquinas:
        visibles[trace_idx[(maq, fase)]] = True

    botones.append(
        dict(
            label=f'Fase {fase}',
            method='update',
            args=[
                {'visible': visibles},
                {'shapes': shapes_por_fase[fase]}
            ]
        )
    )

# Layout final
fig.update_layout(
    updatemenus=[
        dict(
            buttons=botones,
            direction='down',
            x=1.07,
            y=1,
            showactive=True
        )
    ],
    title='Eventos de corriente por maquina',
    shapes=shapes_por_fase['R']  # fase inicial
)
fig.update_yaxes(title_text='Corriente [A]', row=1, col=1)
fig.update_yaxes(title_text='Corriente [A]', row=2, col=1)

fig.show()
